<a href="https://colab.research.google.com/github/ccunique/ds_code_examples/blob/main/Graph/single_giant_graph_in_DGL_Node_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
torch.version.cuda

'11.1'

In [ ]:
!pip install dgl-cu111 -f https://data.dgl.ai/wheels/repo.html

Looking in links: https://data.dgl.ai/wheels/repo.html


# Stochastic Training of GNN for Node Classification on Large Graphs

This tutorial shows how to train a multi-layer GraphSAGE for node classification on Amazon Copurchase Network provided by OGB.  The dataset contains 2.4 million nodes and 61 million edges, hence not fitting in a single GPU.

This tutorial's contents include

* Creating your DGL graph from your own data in other formats such as CSV.
* Training a GNN model with a single machine, a single GPU, on a graph of any size.

## Load Dataset

Although you can directly use the Python package provided by OGB, for demonstration, we will instead manually download the dataset, peek into its contents, and process it with only `numpy`.

In [ ]:
!wget https://snap.stanford.edu/ogb/data/nodeproppred/products.zip

--2021-10-16 07:04:35--  https://snap.stanford.edu/ogb/data/nodeproppred/products.zip
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1480993786 (1.4G) [application/zip]
Saving to: ‘products.zip’

products.zip        100%[===================>]   1.38G  1.63MB/s    in 16m 44s 

2021-10-16 07:21:20 (1.41 MB/s) - ‘products.zip’ saved [1480993786/1480993786]



In [ ]:
!unzip -o products.zip

Archive:  products.zip
   creating: products/
   creating: products/split/
   creating: products/split/sales_ranking/
  inflating: products/split/sales_ranking/test.csv.gz  
  inflating: products/split/sales_ranking/train.csv.gz  
  inflating: products/split/sales_ranking/valid.csv.gz  
   creating: products/processed/
   creating: products/raw/
  inflating: products/raw/node-label.csv.gz  
 extracting: products/raw/num-node-list.csv.gz  
 extracting: products/raw/num-edge-list.csv.gz  
  inflating: products/raw/node-feat.csv.gz  
  inflating: products/raw/edge.csv.gz  
   creating: products/mapping/
  inflating: products/mapping/README.md  
 extracting: products/mapping/labelidx2productcategory.csv.gz  
  inflating: products/mapping/nodeidx2asin.csv.gz  
  inflating: products/RELEASE_v1.txt  


The dataset contains these files:

* `products/raw/edge.csv` (source-destination pairs)
* `products/raw/node-feat.csv` (node features)
* `products/raw/node-label.csv` (node labels)
* `products/raw/num-edge-list.csv` (number of edges)
* `products/raw/num-node-list.csv` (number of nodes)

We will only use the first three CSV files.

In addition, it also contains the following files defining the training-validation-test split in the directory `products/split/sales_ranking`.  All `train.csv`, `valid.csv` and `test.csv` are text files containing the node IDs in the training/validation/test set, one number per line.

<div class="alert alert-info">
    <b>Note:</b> The node IDs should be consecutive integers from 0 to the number of nodes minus 1.  If your node ID is not consecutive or not starting from 0 (e.g., starting from 100000), you need to relabel them yourself.  The <code>astype</code> method in pandas DataFrame can conveniently relabel the IDs by converting the type to <code>"category"</code>.
</div>

In [ ]:
import pandas as pd
edges = pd.read_csv('products/raw/edge.csv.gz', header=None).values
node_features = pd.read_csv('products/raw/node-feat.csv.gz', header=None).values
node_labels = pd.read_csv('products/raw/node-label.csv.gz', header=None).values[:, 0]

# pd.read_csv yields a DataFrame with one column, so we make them one-dimensional arrays.
train_nids = pd.read_csv('products/split/sales_ranking/train.csv.gz', header=None).values[:, 0]
valid_nids = pd.read_csv('products/split/sales_ranking/valid.csv.gz', header=None).values[:, 0]
test_nids = pd.read_csv('products/split/sales_ranking/test.csv.gz', header=None).values[:, 0]

In [ ]:
edges[:5]

array([[     0, 152857],
       [     0,  32104],
       [     0,  23158],
       [     0, 228358],
       [     0, 115556]])

In [ ]:
node_features.shape

(2449029, 100)

In [ ]:
node_labels.shape

(2449029,)

In [ ]:
train_nids.shape

(196615,)

We construct the graph as follows:

In [ ]:
import dgl
import torch

graph = dgl.graph((edges[:, 0], edges[:, 1]))
node_features = torch.FloatTensor(node_features)
node_labels = torch.LongTensor(node_labels)

# Save the graph, features and training-validation-test split for use for future tutorials.
import pickle
with open('data.pkl', 'wb') as f:
    pickle.dump((graph, node_features, node_labels, train_nids, valid_nids, test_nids), f)

Using backend: pytorch


NameError: ignored

In [ ]:
!ls -lh

total 3.3G
-rw-r--r-- 1 root root 1.9G Oct 16 07:28 data.pkl
drwxr-xr-x 6 root root 4.0K May  4  2020 products
-rw-r--r-- 1 root root 1.4G May  4  2020 products.zip
drwxr-xr-x 1 root root 4.0K Oct  8 13:45 sample_data


In [ ]:
# Load the graph back from the file we saved
import dgl
import torch
import numpy as np
import pickle
with open('data.pkl', 'rb') as f:
    graph, node_features, node_labels, train_nids, valid_nids, test_nids = pickle.load(f)

In [ ]:
graph

Graph(num_nodes=2449029, num_edges=61859140,
      ndata_schemes={}
      edata_schemes={})

In [ ]:
graph.edges()

(tensor([     0,      0,      0,  ...,  18965, 162836,  53324]),
 tensor([ 152857,   32104,   23158,  ..., 2449028, 2449028, 2449028]))

In [ ]:
node_features

tensor([[ 0.0319, -0.1959,  0.0520,  ...,  0.0767, -0.3930, -0.0648],
        [-0.0241,  0.6303,  1.0606,  ..., -1.6875,  3.5867,  0.8182],
        [ 0.3327, -0.5586, -0.2886,  ..., -0.3716,  0.2521,  0.0415],
        ...,
        [ 0.1066,  0.2655, -0.0057,  ...,  1.0867,  0.0759, -1.1737],
        [ 0.2497, -0.2574,  0.4123,  ...,  1.5466,  1.0310, -0.2966],
        [ 0.7175, -0.2393,  0.0443,  ..., -1.0132, -0.4141, -0.0823]])

We can see the size of the graph, features, and labels as follows.

In [ ]:
print('Graph')
print(graph)
print('Shape of node features:', node_features.shape)
print('Shape of node labels:', node_labels.shape)

num_features = node_features.shape[1]
num_classes = (node_labels.max() + 1).item()
print('Number of classes:', num_classes)

Graph
Graph(num_nodes=2449029, num_edges=61859140,
      ndata_schemes={}
      edata_schemes={})
Shape of node features: torch.Size([2449029, 100])
Shape of node labels: torch.Size([2449029])
Number of classes: 47


## Define a Data Loader with Neighbor Sampling

### Neighbor sampling overview

The formulation of message passing usually has the following form:

$$
\begin{gathered}
  \boldsymbol{a}_v^{(l)} = \rho^{(l)} \left(
    \left\lbrace
      \boldsymbol{h}_u^{(l-1)} : u \in \mathcal{N} \left( v \right)
    \right\rbrace
  \right)
\\
  \boldsymbol{h}_v^{(l)} = \phi^{(l)} \left(
    \boldsymbol{h}_v^{(l-1)}, \boldsymbol{a}_v^{(l)}
  \right)
\end{gathered}
$$

where $\rho^{(l)}$ and $\phi^{(l)}$ are parameterized functions, and $\mathcal{N}(v)$ represents the set of predecessors (or equivalently *neighbors*) of $v$ on graph $\mathcal{G}$:
$$
\mathcal{N} \left( v \right) = \left\lbrace
  s \left( e \right) : e \in \mathbb{E}, t \left( e \right) = v
\right\rbrace
$$

For instance, to perform a message passing for updating the red node in the following graph:

![Imgur](https://github.com/dglai/KDD20-Hands-on-Tutorial/blob/master/4-large_graph/assets/1.png?raw=1)

One needs to aggregate the node features of its neighbors, shown as green nodes:

![Imgur](https://github.com/dglai/KDD20-Hands-on-Tutorial/blob/master/4-large_graph/assets/2.png?raw=1)

Let's consider how multi-layer message passing works for computing the output of a single node.  In the following text, we refer to the nodes whose outputs GNN will compute as seed nodes.

Consider computing with a 2-layer GNN the output of the seed node 8, colored red, in the following graph:

![Imgur](https://github.com/dglai/KDD20-Hands-on-Tutorial/blob/master/4-large_graph/assets/seed.png?raw=1)

By the formulation:

$$
\begin{gathered}
  \boldsymbol{a}_8^{(2)} = \rho^{(2)} \left(
    \left\lbrace
      \boldsymbol{h}_u^{(1)} : u \in \mathcal{N} \left( 8 \right)
    \right\rbrace
  \right) = \rho^{(2)} \left(
    \left\lbrace
      \boldsymbol{h}_4^{(1)}, \boldsymbol{h}_5^{(1)},
      \boldsymbol{h}_7^{(1)}, \boldsymbol{h}_{11}^{(1)}
    \right\rbrace
  \right)
\\
  \boldsymbol{h}_8^{(2)} = \phi^{(2)} \left(
    \boldsymbol{h}_8^{(1)}, \boldsymbol{a}_8^{(2)}
  \right)
\end{gathered}
$$

We can tell from the formulation that, to compute $\boldsymbol{h}_8^{(2)}$, we need messages from node 4, 5, 7, and 11 (colored green) along the edges visualized below.

![Imgur](https://github.com/dglai/KDD20-Hands-on-Tutorial/blob/master/4-large_graph/assets/3.png?raw=1)

The values of $\boldsymbol{h}_\cdot^{(1)}$ are the outputs from the first GNN layer.  To compute those values for the red and green nodes, we further need to perform message passing on the edges visualized below.

![Imgur](https://github.com/dglai/KDD20-Hands-on-Tutorial/blob/master/4-large_graph/assets/4.png?raw=1)

Therefore, to compute the 2-layer GNN representation of the red node, we need the input features from the red node as well as the green and yellow nodes.  Note that we should take red node's neighbors again for this layer.

You may notice that the procedure which determines computation dependency is in the reverse direction of message aggregation: you start from the layer closest to the output and work backward to the input.

You can also see that computing representation for a small number of nodes often requires input features of a significantly larger number of nodes.  Taking all neighbors for message aggregation is often too costly since the nodes needed would easily cover a large portion of the graph.

Neighbor sampling addresses this issue by selecting a random subset of the neighbors to perform aggregation.  For instance, to compute $\boldsymbol{h}_8^{(1)}$, we can choose to sample 2 neighbors and aggregate.

![Imgur](https://github.com/dglai/KDD20-Hands-on-Tutorial/blob/master/4-large_graph/assets/5.png?raw=1)

Similarly, to compute the red and green nodes' first layer representation, we can also do neighbor sampling that takes 2 neighbors for each node.  Note that we should take the red node's neighbors again for this layer.

![Imgur](https://github.com/dglai/KDD20-Hands-on-Tutorial/blob/master/4-large_graph/assets/6.png?raw=1)

You can see that this method could give us fewer nodes needed for input features.

### Defining neighbor sampler and data loader in DGL

DGL provides useful tools to generate such computation dependencies while iterating over the dataset in minibatches.  For node classification, you can use `dgl.dataloading.NodeDataLoader` for iterating over the dataset, and `dgl.dataloading.MultiLayerNeighborSampler` to generate computation dependencies of the nodes from a multi-layer GNN with neighbor sampling.

The syntax of `dgl.dataloading.NodeDataLoader` is mostly similar to a PyTorch `DataLoader`, with the addition that it needs a graph to generate computation dependency from, a set of node IDs to iterate on, and the neighbor sampler you defined.

Let's consider training a 3-layer GraphSAGE with neighbor sampling, and each node will gather message from 4 neighbors on each layer.  The code defining the data loader and neighbor sampler will look like the following.

In [ ]:
graph

Graph(num_nodes=2449029, num_edges=61859140,
      ndata_schemes={}
      edata_schemes={})

In [ ]:
sampler = dgl.dataloading.MultiLayerNeighborSampler([4, 4, 4])
train_dataloader = dgl.dataloading.NodeDataLoader(
    graph, train_nids, sampler,
    batch_size=1024,
    shuffle=True,
    drop_last=False,
    num_workers=0
)

We can iterate over the data loader we created and see what it gives us.

In [ ]:
example_minibatch = next(iter(train_dataloader))
print(example_minibatch)

[tensor([ 75929,  52592, 116205,  ...,  18550,  11379,   3280]), tensor([ 75929,  52592, 116205,  ..., 182865,  24257,  21553]), [Block(num_src_nodes=34877, num_dst_nodes=15700, num_edges=50967), Block(num_src_nodes=15700, num_dst_nodes=4547, num_edges=15896), Block(num_src_nodes=4547, num_dst_nodes=1024, num_edges=3667)]]


In [ ]:
len(example_minibatch)

3

In [ ]:
example_minibatch[0]

tensor([ 75929,  52592, 116205,  ...,  18550,  11379,   3280])

In [ ]:
example_minibatch[1]

tensor([ 75929,  52592, 116205,  ..., 182865,  24257,  21553])

In [ ]:
example_minibatch[2]

[Block(num_src_nodes=34877, num_dst_nodes=15700, num_edges=50967),
 Block(num_src_nodes=15700, num_dst_nodes=4547, num_edges=15896),
 Block(num_src_nodes=4547, num_dst_nodes=1024, num_edges=3667)]

`NodeDataLoader` gives us three items per iteration.

* The input node list for the nodes whose input features are needed to compute the outputs.
* The output node list whose GNN representation are to be computed.
* The list of computation dependency for each layer.

In [ ]:
input_nodes, output_nodes, bipartites = example_minibatch
print("To compute {} nodes' output we need {} nodes' input features".format(len(output_nodes), len(input_nodes)))

To compute 1024 nodes' output we need 34877 nodes' input features


In [ ]:
# DGL确保块的输出节点将始终出现在输入节点中。如下代码所演示的，在输入节点中，输出节点的ID位于其它节点之前。
torch.equal(input_nodes[:len(output_nodes)], output_nodes)

True

The variable `bipartites` shows how messages aggregate on each layer.  As its name suggests, it is a **list** of bipartite graphs.

So why does DGL return a list of *bipartite* graphs for training a *homogeneous* graph?  The reason is that the number of nodes for input and that for output of a given GNN layer is different.  Take the example above:

![Imgur](https://github.com/dglai/KDD20-Hands-on-Tutorial/blob/master/4-large_graph/assets/6.png?raw=1)

That GNN layer will output the representation of three nodes (two green nodes and one red node), but it will require input from 7 nodes (the green nodes and red node, plus 4 yellow nodes).  Only a bipartite graph can describe such computation:

![](https://github.com/dglai/KDD20-Hands-on-Tutorial/blob/master/4-large_graph/assets/bipartite.png?raw=1)

Minibatch training of GNNs usually involves message passing on such bipartite graphs.

In [ ]:
print(bipartites)

[Block(num_src_nodes=34877, num_dst_nodes=15700, num_edges=50967), Block(num_src_nodes=15700, num_dst_nodes=4547, num_edges=15896), Block(num_src_nodes=4547, num_dst_nodes=1024, num_edges=3667)]


## Defining Model

The model can be written as follows:

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import dgl.nn as dglnn

class SAGE(nn.Module):
    def __init__(self, in_feats, n_hidden, n_classes, n_layers):
        super().__init__()
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.n_classes = n_classes
        self.layers = nn.ModuleList()
        self.layers.append(dglnn.SAGEConv(in_feats, n_hidden, 'mean'))
        for i in range(1, n_layers - 1):
            self.layers.append(dglnn.SAGEConv(n_hidden, n_hidden, 'mean'))
        self.layers.append(dglnn.SAGEConv(n_hidden, n_classes, 'mean'))

    def forward(self, bipartites, x):
        for l, (layer, bipartite) in enumerate(zip(self.layers, bipartites)):
            x = layer(bipartite, x)
            if l != self.n_layers - 1:
                x = F.relu(x)
        return x

You can see that here we are iterating over the pairs of NN module layer and bipartite graphs generated by the data loader.

## Defining Training Loop

The following initializes the model and defines the optimizer.

In [ ]:
model = SAGE(num_features, 128, num_classes, 3).cuda()
opt = torch.optim.Adam(model.parameters())

When computing the validation score for model selection, usually you can also do neighbor sampling.  To do that, you need to define another data loader.

In [ ]:
valid_dataloader = dgl.dataloading.NodeDataLoader(
    graph, valid_nids, sampler,
    batch_size=1024,
    shuffle=False,
    drop_last=False,
    num_workers=0
)

The following is a training loop that performs validation every epoch.  It also saves the model with the best validation accuracy into a file.

In [ ]:
import tqdm
import sklearn.metrics

best_accuracy = 0
best_model_path = 'model.pt'
# for epoch in range(10):
for epoch in range(3):
    model.train()

    with tqdm.tqdm(train_dataloader) as tq:
        for step, (input_nodes, output_nodes, bipartites) in enumerate(tq):
            bipartites = [b.to(torch.device('cuda')) for b in bipartites]
            inputs = node_features[input_nodes].cuda()
            labels = node_labels[output_nodes].cuda()
            predictions = model(bipartites, inputs)

            loss = F.cross_entropy(predictions, labels)
            opt.zero_grad()
            loss.backward()
            opt.step()

            accuracy = sklearn.metrics.accuracy_score(labels.cpu().numpy(), predictions.argmax(1).detach().cpu().numpy())

            tq.set_postfix({'loss': '%.03f' % loss.item(), 'acc': '%.03f' % accuracy}, refresh=False)

    model.eval()

    predictions = []
    labels = []
    with tqdm.tqdm(valid_dataloader) as tq, torch.no_grad():
        for input_nodes, output_nodes, bipartites in tq:
            bipartites = [b.to(torch.device('cuda')) for b in bipartites]
            inputs = node_features[input_nodes].cuda()
            labels.append(node_labels[output_nodes].numpy())
            predictions.append(model(bipartites, inputs).argmax(1).cpu().numpy())
        predictions = np.concatenate(predictions)
        labels = np.concatenate(labels)
        accuracy = sklearn.metrics.accuracy_score(labels, predictions)
        print('Epoch {} Validation Accuracy {}'.format(epoch, accuracy))
        if best_accuracy < accuracy:
            best_accuracy = accuracy
            torch.save(model.state_dict(), best_model_path)

100%|██████████| 39/39 [00:05<00:00,  7.43it/s]


Epoch 0 Validation Accuracy 0.8670243877628868


100%|██████████| 39/39 [00:05<00:00,  7.37it/s]


Epoch 1 Validation Accuracy 0.8683976298858175


100%|██████████| 39/39 [00:05<00:00,  7.42it/s]

Epoch 2 Validation Accuracy 0.8765862218040332


## Offline Inference without Neighbor Sampling

Usually for offline inference it is desirable to aggregate over the entire neighborhood to eliminate randomness introduced by neighbor sampling.  However, using the same methodology in training is not efficient, because there will be a lot of redundant computation.  Moreover, simply doing neighbor sampling by taking all neighbors will often exhaust GPU memory because the number of nodes required for input features may be too large to fit into GPU memory.

Instead, you need to compute the representations layer by layer: you first compute the output of the first GNN layer for all nodes, then you compute the output of second GNN layer for all nodes using the first GNN layer's output as input, etc.  This gives us a different algorithm from what is being used in training.  During training we have an outer loop that iterates over the nodes, and an inner loop that iterates over the layers.  In contrast, during inference we have an outer loop that iterates over the layers, and an inner loop that iterates over the nodes.

If you do not care about randomness too much (e.g., during model selection in validation), you can still use the `dgl.dataloading.MultiLayerNeighborSampler` and `dgl.dataloading.NodeDataLoader` to do offline inference, since it is usually faster for evaluating a small number of nodes.

![Imgur](https://github.com/dglai/KDD20-Hands-on-Tutorial/blob/master/4-large_graph/assets/anim.gif?raw=1)

In [ ]:
def inference(model, graph, input_features, batch_size):
    nodes = torch.arange(graph.number_of_nodes())

    sampler = dgl.dataloading.MultiLayerNeighborSampler([None])  # one layer at a time, taking all neighbors
    dataloader = dgl.dataloading.NodeDataLoader(
        graph, nodes, sampler,
        batch_size=batch_size,
        shuffle=False,
        drop_last=False,
        num_workers=0)

    with torch.no_grad():
        for l, layer in enumerate(model.layers):
            # Allocate a buffer of output representations for every node
            # Note that the buffer is on CPU memory.
            output_features = torch.zeros(
                graph.number_of_nodes(), model.n_hidden if l != model.n_layers - 1 else model.n_classes)

            for input_nodes, output_nodes, bipartites in tqdm.tqdm(dataloader):
                bipartite = bipartites[0].to(torch.device('cuda'))

                x = input_features[input_nodes].cuda()

                # the following code is identical to the loop body in model.forward()
                x = layer(bipartite, x)
                if l != model.n_layers - 1:
                    x = F.relu(x)

                output_features[output_nodes] = x.cpu()
            input_features = output_features
    return output_features

The following code loads the best model from the file saved previously and performs offline inference.  It computes the accuracy on the test set afterwards.

In [ ]:
model.load_state_dict(torch.load(best_model_path))
all_predictions = inference(model, graph, node_features, 8192)

100%|██████████| 299/299 [00:37<00:00,  7.92it/s]


In [ ]:
test_predictions = all_predictions[test_nids].argmax(1)
test_labels = node_labels[test_nids]
test_accuracy = sklearn.metrics.accuracy_score(test_predictions.numpy(), test_labels.numpy())
print('Test accuracy:', test_accuracy)

Test accuracy: 0.7233028375245302


## Conclusion

In this tutorial, you have learned how to train a multi-layer GraphSAGE with neighbor sampling on a large dataset that cannot fit into GPU.  The method you have learned can scale to a graph of any size, and works on a single machine with a single GPU.

## What's next?

The next tutorial will be about training the same GraphSAGE model in an unsupervised manner with link prediction, i.e. predicting whether an edge exist between two nodes.